In [1]:
import torch
import os

from shap_e.models.download import load_model
from shap_e.util.data_util import load_or_create_multimodal_batch
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)

In [28]:
model_path = '../../../content/batch/'

# get full paths to all .obj files in the directory as a list
model_paths = [(model_path + f) for f in os.listdir(model_path) if f.endswith('.obj')]
model_path = model_paths[1]
print(model_path)

../../../content/batch/cone_tall.obj


In [29]:
# This may take a few minutes, since it requires rendering the model twice
# in two different modes.
batch = load_or_create_multimodal_batch(
    device,
    model_path=model_path,
    mv_light_mode="basic",
    mv_image_size=256,
    cache_dir="../../../content/batch/cached",
    verbose=True, # this will show Blender output during renders
)

creating point cloud...
Blender 3.3.18 (hash ce642e790ac6 built 2024-04-15 23:55:12)
/mnt/wslg/runtime-dir/gvfs/ non-existent directory
(  0.0001 sec |   0.0001 sec) Importing OBJ '../../../content/batch/cone_tall.obj'...
  (  0.0116 sec |   0.0115 sec) Parsing OBJ file...
    (  0.0123 sec |   0.0006 sec) Done, loading materials and images...
    (  0.0544 sec |   0.0427 sec) Done, building geometries (verts:33 faces:33 materials: 1 smoothgroups:1) ...
    (  0.1263 sec |   0.1146 sec) Done.
  (  0.1264 sec |   0.1262 sec) Finished importing: '../../../content/batch/cone_tall.obj'
Progress: 100.00%

Fra:1 Mem:15.53M (Peak 16.65M) | Time:00:03.62 | Syncing cone_tall
Fra:1 Mem:15.54M (Peak 16.65M) | Time:00:04.49 | Syncing Light
Fra:1 Mem:15.54M (Peak 16.65M) | Time:00:04.49 | Syncing Light.001
Fra:1 Mem:15.54M (Peak 16.65M) | Time:00:04.49 | Syncing Camera
Fra:1 Mem:15.54M (Peak 16.65M) | Time:00:04.54 | Rendering 1 / 1 samples
Fra:1 Mem:15.05M (Peak 20.54M) | Time:00:10.64 | Compositi

In [ ]:
with torch.no_grad():
    latent = xm.encoder.encode_to_bottleneck(batch)

    render_mode = 'stf' # you can change this to 'nerf'
    size = 128 # recommended that you lower resolution when using nerf

    cameras = create_pan_cameras(size, device)
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

/home/demircantas/sm-thesis/shap-e_construct/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/home/demircantas/sm-thesis/shap-e_construct/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAJdyApNuAZBtAY1qAoxpAYpoAYlmAYZmAYVlAYNlAYRkAYNiAY…

In [ ]:
print(model_path)

../../../content/batch/cone_tall.obj


In [ ]:
# Save latent to disk using model_path as base but change extension to .pt

torch.save(latent, model_path.replace(".obj", ".pt"))

# torch.save(latent, "../../../content/batch/latent_cone_short.pt")

# Save images to disk using model_path as base but change extension to .gif
images[0].save(model_path.replace(".obj", ".gif"), save_all=True, append_images=images[1:], loop=0, duration=1000)

# images[0].save("../../../content/batch/cone_short.gif", save_all=True, append_images=images[1:], loop=0, duration=1000)



In [ ]:
latent.shape

torch.Size([1, 1048576])